# 1. EDA
En este apartado se pretende analizar los datasets para poder enfocar mejor la limpieza

## 1.1 Importación y carga de datos
Debemos declarar las librerías que usamos y leer el correspondiente archivo de datos

In [1]:
# Importación de librerías
import pandas as pd

# Lectura dataset
df = pd.read_csv('../IncidentesSeguridadSucio.csv')

## 1.2 Configuración de Pandas
Para poder leer bien los resultados de las ejecuciones, vamos a configurar tanto el número máximo de columnas como el número máximo de filas

In [2]:
# Número máximo de filas a mostrar
pd.set_option('display.max_rows', None)

# Número máximo de columnas a mostrar
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 2000)

## 1.3 Descripción general del dataset
Para poder conocer ciertas características relevantes del dataset, como el número de instancias (filas) y características (columnas) procederemos a usar diferentes funciones de Pandas


In [3]:
# Descripción de parámetros generales -> count, mean, std, min, 25%, 50%, 75%, max
print(df.describe())

# Número de filas y columnas del dataset
print("\n")
print("Número de filas: ", df.shape[0])
print("Número de columnas: ", df.shape[1])
print("\n")

# Para saber el tipo de variable de cada columna
print(df.info())

                ID  AreaRecreativaID
count  5005.000000      5.005000e+03
mean   2501.294505      1.530310e+06
std    1443.397288      2.791697e+06
min       1.000000      1.665700e+04
25%    1252.000000      1.742100e+04
50%    2502.000000      1.811800e+04
75%    3751.000000      4.513510e+05
max    5000.000000      9.536964e+06


Número de filas:  5005
Número de columnas:  5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5005 entries, 0 to 5004
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                5005 non-null   int64 
 1   FECHA_REPORTE     5005 non-null   object
 2   TIPO_INCIDENTE    5005 non-null   object
 3   GRAVEDAD          5005 non-null   object
 4   AreaRecreativaID  5005 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 195.6+ KB
None


## 1.4 Observación inicial del dataset
Vamos a mostrar 30 entradas para poder observar cómo es realmente por dentro el dataset

In [4]:
# Mostramos las 10 primeras filas, 10 filas aleatorias y las 10 últimas
print("   ----------    10 primeras filas    ----------    ")
print("\n")
print(df.head(10))
print("\n")
print("   ----------    10 filas aleatorias    ----------    ")
print("\n")
print(df.sample(10))
print("\n")
print("   ----------    10 últimas filas    ----------    ")
print("\n")
print(df.tail(10))

   ----------    10 primeras filas    ----------    


   ID FECHA_REPORTE TIPO_INCIDENTE GRAVEDAD  AreaRecreativaID
0   1    2024/06/13           Robo     Alta             17538
1   2    07-16-2024          Cáídá     Baja             16724
2   3    2021/03/09          Caída  Crítica             18191
3   4    10-15-2023      Accidente    Media             52162
4   5    2022/06/25      Accidente     Alta             17397
5   6    08/06/2020           Robo     Baja             17260
6   7    2023/08/23     Vándálísmó  Crítica           7928031
7   8    2022-01-15      Accidente  Crítica             17278
8   9    27-08-2024     Vandalismo    Media             16729
9  10    04-11-2020      Accídénté  Crítica             17203


   ----------    10 filas aleatorias    ----------    


        ID FECHA_REPORTE    TIPO_INCIDENTE GRAVEDAD  AreaRecreativaID
3327  3328    2022-10-12              Robo     Baja             18316
4668  4669    12/02/2022        Vandalismo     Baja             

## 1.5 Revisión de valores nulos
Como ya se ha visto en el anterior apartado (info), podemos observar los valores nulos de esa forma. Pero se puede observar de una forma más visual en esta sección y, además, hay que tener en cuenta valores como el cero que también pueden considerarse nulos.

In [5]:
# Para poder saber el número de valores faltantes
print("    ----------    Valores faltantes    ----------    ")
print("\n")
print(df.isnull().sum())
print("\n")

# Para poder saber el número de ceros en cada columna
print("    ----------    Valores cero    ----------    ")
print("\n")
print((df == 0).sum())

    ----------    Valores faltantes    ----------    


ID                  0
FECHA_REPORTE       0
TIPO_INCIDENTE      0
GRAVEDAD            0
AreaRecreativaID    0
dtype: int64


    ----------    Valores cero    ----------    


ID                  0
FECHA_REPORTE       0
TIPO_INCIDENTE      0
GRAVEDAD            0
AreaRecreativaID    0
dtype: int64


No encontramos valores nulos, por lo que no se hará limpieza de esto

## 1.6 Identificación de fechas no estandarizadas
Se deben identificar las fechas que no se encuentran en el formato adecuado para MongoDB (DD/MM/YYYY)

In [6]:
# Vamos a mostrar algunas fechas para poder observar en qué formato están
print("   ----------    Fechas    ----------    ")
print("\n")
print(df['FECHA_REPORTE'].sample(20))

   ----------    Fechas    ----------    


4537    03-10-2020
2077    2022-09-03
311     2020-11-29
2910    2022/05/03
2125    2023/05/23
1987    09-01-2021
1071    08-10-2023
3429    07-14-2022
1021    03-23-2020
4967    2023-05-11
3809    2022-11-22
4068    04-19-2020
2497    07-03-2020
3450    2023-03-26
643     27/01/2024
843     18-07-2023
507     2020-09-26
781     01-18-2024
2888    21-11-2019
2316    2021/10/25
Name: FECHA_REPORTE, dtype: object


Como se puede apreciar, las fechas se encuentran en diversos formatos que deben ser homogeneizados

## 1.7 Identificación de registros duplicados
Debemos validar que no existen filas iguales que ensucien el dataset, sobre todo estando pendiente de duplicaciones de la clave primaria

In [7]:
# Ver las filas duplicadas
print("   ----------    Filas duplicadas    ----------    ")
print("\n")
print(df[df.duplicated()])
print("\n")

# Número de filas duplicadas
print("Número de filas duplicadas: ", df.duplicated().sum())
print("\n")

# Filtrar filas que tienen el mismo ID
duplicados = df.groupby('ID').filter(lambda x: len(x) > 1)

# Ordenar por NIF para que las filas con el mismo NIF se visualicen una encima de la otra
duplicados = duplicados.sort_values(by='ID')

# Mostrar las filas con la misma PK
print("   ----------    Filas con el mismo ID    ----------    ")
print("\n")
print(duplicados)
print("\n")
print("Número de filas con el mismo ID: ", duplicados.shape[0])

   ----------    Filas duplicadas    ----------    


        ID FECHA_REPORTE TIPO_INCIDENTE GRAVEDAD  AreaRecreativaID
5000  3501    07/07/2020     Vandalismo    Media           7125670
5001  4346    27/08/2020     Vándálísmó     Baja             17297
5002  3958    16/03/2020          Caída     Baja             18567
5003  3353    19-08-2020          Cáídá     Baja             18590
5004  1321    2023/12/31      Accidente     Baja             17931


Número de filas duplicadas:  5


   ----------    Filas con el mismo ID    ----------    


        ID FECHA_REPORTE TIPO_INCIDENTE GRAVEDAD  AreaRecreativaID
1320  1321    2023/12/31      Accidente     Baja             17931
5004  1321    2023/12/31      Accidente     Baja             17931
5003  3353    19-08-2020          Cáídá     Baja             18590
3352  3353    19-08-2020          Cáídá     Baja             18590
3500  3501    07/07/2020     Vandalismo    Media           7125670
5000  3501    07/07/2020     Vandalismo    Media

Existen filas iguales que no difieren en nada entre ellas, por lo que pueden ser eliminadas sin problemas

## 1.8 Búsqueda de errores tipográficos
Hay ciertos atributos de texto que pueden contar con determinados errores tipográficos que deben ser solucionados, como los nombres de áreas, juegos, usuarios y ubicaciones

En este dataset no encontramos ningún atributo similar, por lo que este paso no se realizará

## 1.9 Identificación de valores enum fuera de campo
Hay ciertos atributos que solo deben poseer ciertos valores (como Operativo-NoOperativo). Hace falta identificar aquellos valores de ese campo fuera de la norma

In [8]:
# Muestra todos los valores distintos de la columna 'TIPO_INCIDENTE'
print("   ----------    Valores distintos de la columna 'TIPO_INCIDENTE'    ----------    ")
print("\n")
print(df['TIPO_INCIDENTE'].unique())
print("\n")

# Muestra todos los valores distintos de la columna 'GRAVEDAD'
print("   ----------    Valores distintos de la columna 'GRAVEDAD'    ----------    ")
print("\n")
print(df['GRAVEDAD'].unique())

   ----------    Valores distintos de la columna 'TIPO_INCIDENTE'    ----------    


['Robo' 'Cáídá' 'Caída' 'Accidente' 'Vándálísmó' 'Vandalismo' 'Accídénté'
 'Daño estructural' 'Dáñó éstrúctúrál' 'Róbó']


   ----------    Valores distintos de la columna 'GRAVEDAD'    ----------    


['Alta' 'Baja' 'Crítica' 'Media']


Todos los Enums poseen valores acordes con lo esperado, por lo que no se va a realizar una limpieza como tal de ellos. Aunque se debe limpiar los valores de TIPO_INCIDENTE que poseen tildes raras

## 1.10 Validación de las coordenadas y otros campos geoespaciales
Hay algunas veces en las que los códigos postales no respetan la identificación de Madrid (280..) o el formato, ya sean códigos postales u otros atributos de geolocalización

En este dataset no encontramos valores de localización, por lo que este paso no se realizará

## 1.11 Identificación de unidades de medida en un formato no estandarizado
Se deben identificar las filas que no posean un formato estándar

En este dataset no encontramos valores de unidades de medida, por lo que este paso no se realizará

## 1.12 Otros atributos a corregir
En esta sección se mencionarán aquellos atributos que también deban ser limpiados por errores

### 1.12.1 Validación de IDs
Para este paso debemos comprobar que todos los IDs son enteros, ya que en el paso de los duplicados ya hemos comprobado esto

In [9]:
# Debemos validar que todos los IDs son únicos y enteros
print("   ----------    Validación de IDs    ----------    ")
print("\n")
print('El ID es un entero: ', df['ID'].apply(lambda x: x.is_integer()).all())

   ----------    Validación de IDs    ----------    


El ID es un entero:  True


Todos los IDs son enteros, por lo que no se precisará su limpieza

### 1.12.2 Consideraciones extras de Fechas
Cabe recalcar que si el campo de fechas no posee un formato esperado, las funciones de limpieza darán error, por lo que no hace falta hacer ahora las comprobaciones de sus valores para determinar si son correctos.

### 1.12.3 Consideraciones extras de AreaRecreativaID
Se ha observado que los IDs presentes en el dataset de juegos comparten formato con estos y que, en este dataset, no hay nulos, por lo que lo único que validaremos es que todos los datos de esta columna están presentes en el dataset de áreas

In [10]:
# Comprobar que todos los valores de la columna 'AreaRecreativaID' existen en la columna 'ID' del dataset 'AreasSucio.csv'
areas = pd.read_csv('../AreasSucio.csv')
print("   ----------    Comprobación de valores en 'AreaRecreativaID'    ----------    ")
print("\n")
print("Todos los valores se encuentran en el otro dataset: ", df['AreaRecreativaID'].isin(areas['ID']).all())

   ----------    Comprobación de valores en 'AreaRecreativaID'    ----------    


Todos los valores se encuentran en el otro dataset:  True


# 2. Limpieza de los datasets
En este apartado se realizará la limpieza según la información obtenida en el análisis exploratorio de datos:
- Se debe corregir las fechas y dejarlas en un formato homogéneo
- Se deben solucionar las filas repetidas, eliminando una de las dos iguales
- Se deben limpiar los errores tipográficos de TIPO_INCIDENTE

## 2.1 Limpieza de FECHA_REPORTE
Como en otros datasets, debemos dejar todas las fechas en formato apto para MongoDB (DD/MM/YYYY)

In [11]:
# Leemos toda la columna de fechas y almacenamos todas las fechas en una lista
fechas = df['FECHA_REPORTE'].tolist()

# Ahora, pasamos por todas las fechas y corregimos las fechas que están mal escritas
fechas_corregidas = []
año_al_final = False

for fecha in fechas:
    # Si fecha contiene un guion, hacer fecha.split('-') y si fecha contiene una barra, hacer fecha.split('/')
    if '-' in fecha:
        fecha_split = fecha.split('-')
    elif '/' in fecha:
        fecha_split = fecha.split('/')
    # Condiciones para saber el día, mes y año
    # Encontrar el año
    if (int(fecha_split[0])>31):
        año = fecha_split[0]
        año_al_final = False
    elif (int(fecha_split[2])>31):
        año = fecha_split[2]
        año_al_final = True
    else:
        print("No se ha encontrado el año")
        fechas_corregidas.append("fecha incorrecta")
        break
    # Encontrar el día y el mes
    # Si el año está al final, no comprobamos el primer caracter
    if año_al_final:
        if (int(fecha_split[0])<32 and int(fecha_split[0])>12):
            dia = fecha_split[0]
            mes = fecha_split[1]
        elif (int(fecha_split[1])<32 and int(fecha_split[1])>12):
            dia = fecha_split[1]
            mes = fecha_split[0]
        # Si no hay ningun número entre el 13 y el 31, se asume que el mes es el segundo siempre y el día es el primero
        else:
            dia = fecha_split[0]
            mes = fecha_split[1]
    else:
        if (int(fecha_split[1])<32 and int(fecha_split[1])>12):
            dia = fecha_split[1]
            mes = fecha_split[2]
        elif (int(fecha_split[2])<32 and int(fecha_split[2])>12):
            dia = fecha_split[2]
            mes = fecha_split[1]
        # Si no hay ningun número entre el 13 y el 31, se asume que el mes es el segundo siempre y el día es el tercero
        else:
            dia = fecha_split[2]
            mes = fecha_split[1]
    fechas_corregidas.append(dia+"/"+mes+"/"+año)
    
# Ahora, cambiamos toda la columna de FECHAS_INTERVENCION por las fechas corregidas
df['FECHA_REPORTE'] = fechas_corregidas

# Mostramos las fechas corregidas
print("   ----------    Columna de fechas corregidas    ----------    ")
print("\n")
print(df['FECHA_REPORTE'].sample(20))

   ----------    Columna de fechas corregidas    ----------    


264     17/11/2019
1666    21/06/2021
3288    05/04/2024
1010    19/11/2021
2287    17/02/2024
137     04/10/2021
1280    06/10/2020
2428    22/09/2022
1286    12/02/2022
1188    04/08/2024
4645    04/11/2022
1742    04/12/2020
2131    17/11/2021
2151    12/01/2022
2926    19/05/2024
1310    20/05/2022
4844    28/07/2023
4387    17/01/2020
4600    10/05/2022
839     25/07/2020
Name: FECHA_REPORTE, dtype: object


## 2.2 Limpieza de filas repetidas
Se deben limpiar las filas iguales, eliminando una de las dos repetidas

In [12]:
# Filtrar filas que tienen el mismo ID
duplicados = df.groupby('ID').filter(lambda x: len(x) > 1)

# Ordenar por NIF para que las filas con el mismo ID se visualicen una encima de la otra
duplicados = duplicados.sort_values(by='ID')

# Mostrar las filas con la misma PK
print("   ----------    Filas con el mismo ID    ----------    ")
print("\n")
print(duplicados)
print("\n")
print("Número de filas con el mismo ID: ", duplicados.shape[0])
print("\n")

# Limpiamos una de las dos filas con el mismo ID de duplicados
df.drop_duplicates(subset='ID', keep='first', inplace=True)

# Mostramos el resultado por pantalla
print("   ----------    Filas con el mismo ID después de limpiar    ----------    ")
print("\n")
print(df[df.duplicated()])
print("\n")
# Número de filas duplicadas
print("Número de filas duplicadas: ", df.duplicated().sum())
print("\n")

# Mostramos, para comprobar, 2 filas de las que antes estaban duplicadas
print("   ----------    2 filas de las que antes estaban duplicadas    ----------    ")
print("\n")
print(df.loc[df['ID'] == 1321])
print(df.loc[df['ID'] == 3353])

   ----------    Filas con el mismo ID    ----------    


        ID FECHA_REPORTE TIPO_INCIDENTE GRAVEDAD  AreaRecreativaID
1320  1321    31/12/2023      Accidente     Baja             17931
5004  1321    31/12/2023      Accidente     Baja             17931
5003  3353    19/08/2020          Cáídá     Baja             18590
3352  3353    19/08/2020          Cáídá     Baja             18590
3500  3501    07/07/2020     Vandalismo    Media           7125670
5000  3501    07/07/2020     Vandalismo    Media           7125670
3957  3958    16/03/2020          Caída     Baja             18567
5002  3958    16/03/2020          Caída     Baja             18567
5001  4346    27/08/2020     Vándálísmó     Baja             17297
4345  4346    27/08/2020     Vándálísmó     Baja             17297


Número de filas con el mismo ID:  10


   ----------    Filas con el mismo ID después de limpiar    ----------    


Empty DataFrame
Columns: [ID, FECHA_REPORTE, TIPO_INCIDENTE, GRAVEDAD, AreaRecreativa

## 2.3 Limpieza de TIPO_INCIDENTE
En esta columna Enum, tenemos ciertos valores con tildes cuando no deberían poseerlas. Hace falta limpiarlas

In [13]:
# Limpiamos la columna de 'TIPO_INCIDENTE' para que no haya tildes en lugares raros
df['TIPO_INCIDENTE'] = df['TIPO_INCIDENTE'].str.replace('Cáídá', 'Caída')
df['TIPO_INCIDENTE'] = df['TIPO_INCIDENTE'].str.replace('Róbó', 'Robo')
df['TIPO_INCIDENTE'] = df['TIPO_INCIDENTE'].str.replace('Accídénté', 'Accidente')
df['TIPO_INCIDENTE'] = df['TIPO_INCIDENTE'].str.replace('Vándálísmó', 'Vandalismo')
df['TIPO_INCIDENTE'] = df['TIPO_INCIDENTE'].str.replace('Dáñó éstrúctúrál', 'Daño estructural')

# Muestra todos los valores distintos de la columna 'TIPO_INCIDENTE'
print("   ----------    Valores distintos de la columna 'TIPO_INCIDENTE'    ----------    ")
print("\n")
print(df['TIPO_INCIDENTE'].unique())

   ----------    Valores distintos de la columna 'TIPO_INCIDENTE'    ----------    


['Robo' 'Caída' 'Accidente' 'Vandalismo' 'Daño estructural']


## 2.4 Mostrar Dataset Limpio y guardar CSV
Vamos a mostrar algunas filas del dataset limpio para validar que todo está OK y guardamos el Dataset

In [14]:
# Enseñamos 20 filas aleatorias para ver cómo quedan
print("   ----------    10 filas aleatorias    ----------    ")
print("\n")
print(df.sample(10))
print("\n")

# Guardamos el dataset limpio
df.to_csv('../IncidentesSeguridadLimpio.csv', index=False)
print("\n")
print("Dataset limpio guardado")

   ----------    10 filas aleatorias    ----------    


        ID FECHA_REPORTE    TIPO_INCIDENTE GRAVEDAD  AreaRecreativaID
1828  1829    28/05/2021             Caída    Media             17084
1782  1783    19/06/2020        Vandalismo  Crítica             16942
1846  1847    18/11/2020         Accidente  Crítica             17608
1723  1724    29/04/2020        Vandalismo    Media             16867
4984  4985    05/11/2020  Daño estructural     Alta             17521
1283  1284    05/09/2020         Accidente    Media             17379
1607  1608    03/02/2023         Accidente     Alta             17572
2374  2375    19/01/2023         Accidente     Baja             18539
1437  1438    11/03/2024             Caída  Crítica             18540
1984  1985    12/06/2020              Robo  Crítica             52115




Dataset limpio guardado
